In [1]:
import numpy as np
import pandas as pd

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
# coloums to keep
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace=True)

In [7]:
movies.duplicated().sum()

0

In [8]:
movies.iloc[0].genres 

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [10]:
movies['genres']=movies['genres'].apply(convert)

In [11]:
import ast
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else: 
            break
    return L

In [12]:
movies['cast']=movies['cast'].apply(convert3)

In [13]:
movies['keywords']=movies['keywords'].apply(convert)

In [14]:
import ast
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [15]:
movies['crew']=movies['crew'].apply(fetch_director)

In [16]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [17]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [18]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [19]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [20]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [21]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [22]:
!pip install nltk

In [23]:
import nltk

In [24]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [25]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [26]:
new['tags']=new['tags'].apply(stem)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [28]:
vectors=cv.fit_transform(new['tags']).toarray()

In [29]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [31]:
#applying stemming
#['loved','love','loving']
#converted to ['love','love','love']
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity=cosine_similarity(vectors)

In [33]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1214, 0.28676966733820225),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957)]

In [34]:
def recommend(movie):
    # Check if the movie exists in the dataset
    if movie in new['title'].values:
        # Get the index of the movie
        movie_index = new[new['title'] == movie].index[0]
        
        # Calculate the distances
        distances = similarity[movie_index]
        
        # Get a list of similar movies, excluding the first one (which is the movie itself)
        movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

        # Print the indices of the recommended movies
        for i in movies_list:
            print(new.iloc[i[0]].title)
    else:
        print(f"Movie '{movie}' not found in the dataset.")


In [35]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [36]:
import pickle


In [37]:
pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))

In [38]:
pickle.dump(similarity,open('similarity.pkl','wb'))